In [ ]:
!pip install --ignore-installed PyYAML
exit()

In [ ]:
%pip install moviepy

In [ ]:
!pip install pydub
from pydub import AudioSegment
!pip3 install google-cloud --upgrade
!pip3 install google-cloud-speech --upgrade
import io
import os
import wave
import moviepy.editor as mp
from google.cloud import speech_v1 as speech
from google.cloud import storage
!pip install datasets
!pip install transformers
!pip install torch
import torch
from transformers import LEDForConditionalGeneration, LEDTokenizer

In [ ]:
my_clip = mp.VideoFileClip(r"C:/Users/HP/Desktop/Project/Video_Output/testout_simple.mp4")
my_clip.audio.write_audiofile(r"C:/Users/HP/Desktop/Project/my_result7.wav")

filepath = "C:/Users/HP/Desktop/Project/"     #Input audio file path
filename = "my_result7.wav"
output_filepath = "C:/Users/HP/Desktop/Project/Document/" #Final transcript path
bucketname = "test-summary-567" #Name of the bucket created in the step before
%env GOOGLE_APPLICATION_CREDENTIALS=C:/Users/HP/Desktop/Project/smiling-pact-324809-5103f4e831bf.json
def mp3_to_wav(audio_file_name):
    if audio_file_name.split('.')[1] == 'mp3':    
        sound = AudioSegment.from_mp3(audio_file_name)
        audio_file_name = audio_file_name.split('.')[0] + '.wav'
        sound.export(audio_file_name, format="wav")


def stereo_to_mono(audio_file_name):
    sound = AudioSegment.from_wav(audio_file_name)
    sound = sound.set_channels(1)
    sound.export(audio_file_name, format="wav")

def frame_rate_channel(audio_file_name):
    with wave.open(audio_file_name, "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        channels = wave_file.getnchannels()
        return frame_rate,channels


def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)


def delete_blob(bucket_name, blob_name):
    """Deletes a blob from the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.delete()


def google_transcribe(audio_file_name):
    
    file_name = filepath + audio_file_name
    
    mp3_to_wav(file_name)

    # The name of the audio file to transcribe
    
    frame_rate, channels = frame_rate_channel(file_name)
    
    if channels > 1:
        stereo_to_mono(file_name)
    
    bucket_name = bucketname
    source_file_name = filepath + audio_file_name
    destination_blob_name = audio_file_name
    
    upload_blob(bucket_name, source_file_name, destination_blob_name)
    
    gcs_uri = 'gs://' + bucketname + '/' + audio_file_name
    transcript = ''
    
    client = speech.SpeechClient()
    audio = speech.RecognitionAudio(uri=gcs_uri)

    config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=frame_rate,
    language_code='en-US')

    # Detects speech in the audio file
    operation = client.long_running_recognize(config=config, audio=audio)
    # operation = client.recognize(config, audio)
    response = operation.result(timeout=10000)

    for result in response.results:
        transcript += result.alternatives[0].transcript
    
    #delete_blob(bucket_name, destination_blob_name)
    return transcript

def write_transcripts(transcript_filename,transcript):
    f= open(output_filepath + transcript_filename,"w+")
    f.write(transcript)
    f.close()

def transcribe_audio():
  # use GCP to transcribe audio data
  transcript = google_transcribe(filename)
  transcript_filename = filename.split('.')[0] + '.txt'
  write_transcripts(transcript_filename,transcript)
  return transcript
  
  #LONG_ARTICLE="""Well, last lecture, you saw the utility of using data types that you can create on your own. This time, we're going to take a look at the process of creating our own data types. Let's start with a quick overview of what we need to do. This is the last in our building blocks for programming where we've already considered all these basic things up through functions and modules. And now we're going to talk about programming with objects, and this is going to give you the ability to bring your own abstractions to life. So in general, this process is called object-oriented programming or OOP. It's where we create your own data types,create our own data types, use them in our programs, and creating the data type and using it in our programs is really where we talk about these objects. Where an object is something that holds the data type value. So our variable names refer to objects and objects hold data type value. So for example when we worked with color,we had our set of values and our operations were get red component and brighten and so forth. There were objects that hold these data type values that we manipulated. And we looked at picture, where the set of values was to the array of colors. And then we have objects that hold those values in strings and so forth. And really what we usually want to do is create what's called an abstract data type, where it's the data type whose representation is hidden from the client. The impact of that is that we can use abstract data types without knowing much about the implementation details, and these are examples of abstract data types. Last time we used those we wrote programs that made use of them without knowing anything about the implementation details. We talk about client programs for several abstract data types in the previous lecture. Now we're going to talk about how to create and how to implement your own abstract data types. So, to create it what we have to do is provide some code, and very specific task that we need to do with this code. So one thing we have to do is define the set of values. And we do that with what's called instance variables. And then we have to implement operations on those values, and those are called methods. And we also have to be able to create and initialize the values of new objects. And we do that with special methods called constuctors. So what are instance variables? Well, they're declarations that associate variable names with types, as usual. But the set of type values of the set of instance variables is the set of values for the data type. And we'll see that in examples,it's very simple. Methods or like static methods, and they can refer to instance variables, and again, it's very similar to static methods, but with a profound difference, and you'll see it with some simple examples in the beginning. And a constructor is also like a method that has the same name as the type. And it doesn't actually return,it doesn't have a return type because it's going to return reference to an object of the data type. Those are the basic steps that we have to do and we look at this in much more detail in just a minute. The constructor, by the way,is invoked by new keyword in Java and it returns an object of the type. So, in Java, all of this put together is known as a class. And so, a Java class that we consider are all going to have this basic structure. For every data type that we implement,we'll look at exactly in this framework. We'll look at what are instance variables,what's the set of data type values. What are the constructors? How do you create an object and initialize it's value? What are the methods? What are the operations that we can perform on the values? And then also,we always add a test client or include a test client in every Java data type that we implement. So that's the process that we're going to go through for several examples in this lecture. So here's just an anatomy of a class with real code and then we'll look at each piece of this code in detail in just in a minute. So we put the whole class in a text file named Charge.Java, and that class name, we go public class and give the class name, that's going to be the same as the file name. And then inside the braces go our four components, the instance variables, the constructor,the methods and test client. Now main is a static method, and you can have other static methods in a class, so we're familiar with that. These methods here are not static,and we'll talk about what that means. That's really the essence of what goes on with object oriented programming, and that's really what this lecture is going to be all about."""
from transformers import LEDForConditionalGeneration, LEDTokenizer
import torch
def generate_summary():
  tokenizer = LEDTokenizer.from_pretrained("patrickvonplaten/led-large-16384-pubmed")
  LONG_ARTICLE=transcribe_audio()
  input_ids = tokenizer(LONG_ARTICLE, return_tensors="pt").input_ids.to("cuda")
  global_attention_mask = torch.zeros_like(input_ids)
  # set global_attention_mask on first token
  global_attention_mask[:, 0] = 1

  model = LEDForConditionalGeneration.from_pretrained("patrickvonplaten/led-large-16384-pubmed", return_dict_in_generate=True).to("cuda")

  sequences = model.generate(input_ids, global_attention_mask=global_attention_mask).sequences

  summary = tokenizer.batch_decode(sequences)
  replaced_tags = data["summary"].replace("<s>","<p>").replace("</s>","</p>")
  # print(replaced_tags)
  print(summary)
  # return {"summary":summary[0]}

generate_summary()
